---
title: "Blog"
echo: false
jupyter: python3
section-divs: false
page-layout: full
---

In [ ]:
#| output: asis
from datetime import datetime as dt
from collections import defaultdict
import re
import glob
import json
from string import Template
from IPython.display import display, HTML

# Define functions for rendering blog entries
def make_html_link(nb, attributes, show_topic_tag=False, show_date_tag=False):
    tags = []
    if show_topic_tag:
        tags.append(f'<span class="tag" style="background:#eee; border-radius:4px; padding:0.1em 0.4em; font-size:0.8em;">{attributes["topic"]}</span>')
    if show_date_tag:
        tags.append(f'<span class="tag" style="background:#f5f5f5; border-radius:4px; padding:0.1em 0.4em; font-size:0.8em;">{attributes["date"]}</span>')
    if attributes.get("venue"):
        tags.append(f'<span class="tag" style="background:#ddd; border-radius:4px; padding:0.1em 0.4em; font-size:0.8em;">{attributes["venue"]}</span>')
    tags_html = " ".join(tags)

    s = f"""
    <div class="blog-entry" style="margin-bottom: 1.5em;">
        <div class="blog-entry-flex" style="display:flex; gap: 2em;">
            <div class="blog-image" style="flex-shrink:0; width: 30%; aspect-ratio: 4 / 3; overflow: hidden;">
                <img src="../images/blogs/{attributes['image']}" style="width: 100%; height: 100%; object-fit: cover; object-position: top left;">
            </div>
            <div style="display: flex; flex-direction: column; gap: 0.3em;">
                <a href="{nb}" class="blog-title"><strong>{attributes['title']}</strong></a>
                <div>{tags_html}</div>
                <p>{attributes['preview']}</p>
            </div>
        </div>
    </div>
    """
    return Template(s).substitute(nb=nb, **attributes)

def make_card_html_link(nb, attributes):
    tags_html = f"""
    <div class="blog-tags">
        <span class="tag">{attributes['topic']}</span>
    """

    if attributes.get("venue"):
        tags_html += f'<span class="tag">{attributes["venue"]}</span>'

    tags_html += "</div>"

    card_template = Template(f"""
    <a href="$nb" class="blog-card" style="border:1px solid #ccc; border-radius:1em; overflow:hidden; text-decoration:none; margin-bottom:1em;">
        <div class="card-image" style="aspect-ratio:16/9; overflow:hidden;">
            <img src="../images/blogs/$image" alt="$title" style="width:100%; height:100%; object-fit:cover;">
        </div>
        <div class="card-content" style="padding:1em;">
            {tags_html}
            <h4 class="card-title" style="margin:0.5em 0;">$title</h4>
            <p class="card-text">$preview</p>
        </div>
        <div class="card-footer" style="display:flex; justify-content:space-between; padding:0 1em 1em 1em;">
            <span class="card-date">{attributes['date']}</span>
            <span class="card-icon">➤</span>
        </div>
    </a>
    """)
    return card_template.substitute(nb=nb, **attributes)

# Load and parse metadata
D = {}
for nb in glob.glob("../notebooks/*.ipynb"):
    with open(nb) as f:
        metadata = json.load(f)["cells"][0]
    SIMPLE_ATTRIBUTE = re.compile(r"(\w+) ?: ?(.+)")
    simple_attributes = [re.match(SIMPLE_ATTRIBUTE, line) for line in metadata["source"]]
    D[nb] = defaultdict(str, (attribute.groups() for attribute in simple_attributes if attribute is not None))

# Sort and group
sorted_by_date = sorted(D, key=lambda x: dt.strptime(D[x]["date"], "%Y-%m-%d"), reverse=True)
topics = sorted(set(D[nb]["topic"] for nb in D))

# Build the HTML
html = """
<div class="controls" style="margin-bottom: 3em; margin-top: -1em; text-align: center;">
    <label><strong>Sort by:</strong></label>
    <select id="sort-mode" onchange="updateView()">
        <option value="date" selected>Date</option>
        <option value="topic">Type</option>
    </select>
    <label style="margin-left: 1em;"><strong>View:</strong></label>
    <select id="view-mode" onchange="updateView()">
        <option value="list">List</option>
        <option value="card" selected>Cards</option>
    </select>
</div>
<div id="blog-container">
</div>
<script>
function updateView() {
    const sortMode = document.getElementById("sort-mode").value;
    const viewMode = document.getElementById("view-mode").value;
    const allSections = document.querySelectorAll('.blog-section');
    allSections.forEach(section => section.style.display = 'none');
    const selected = document.getElementById(`by-${sortMode}-${viewMode}`);
    if (selected) selected.style.display = 'block';
}
document.addEventListener("DOMContentLoaded", updateView);
</script>
"""

# Sections
sections = []

# by-date-list
sections.append('<div class="blog-section" id="by-date-list" style="display:none;">')
for nb in sorted_by_date:
    sections.append(make_html_link(nb, D[nb], show_topic_tag=True, show_date_tag=True))
sections.append('</div>')

# by-date-card
sections.append('<div class="blog-section" id="by-date-card"><div class="card-container" style="display:grid; grid-template-columns: repeat(auto-fill, minmax(300px, 1fr)); gap: 1.5em;">')
for nb in sorted_by_date:
    sections.append(make_card_html_link(nb, D[nb]))
sections.append('</div></div>')

# by-topic-list
sections.append('<div class="blog-section" id="by-topic-list" style="display:none;">')
for topic in topics:
    sections.append(f"<h2>{topic}</h2>")
    for nb in sorted_by_date:
        if D[nb]["topic"] == topic:
            sections.append(make_html_link(nb, D[nb], show_topic_tag=True, show_date_tag=True))
sections.append('</div>')

# by-topic-card
sections.append('<div class="blog-section" id="by-topic-card" style="display:none;">')
for topic in topics:
    sections.append(f"<h2>{topic}</h2><div class='card-container' style='display:grid; grid-template-columns: repeat(auto-fill, minmax(300px, 1fr)); gap: 1.5em;'>")
    for nb in sorted_by_date:
        if D[nb]["topic"] == topic:
            sections.append(make_card_html_link(nb, D[nb]))
    sections.append("</div>")
sections.append('</div>')

# Display all
display(HTML(html + "".join(sections)))